In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame, Index, MultiIndex
import matplotlib.pyplot as plt
%matplotlib inline

首先导入文件，并查看数据样本

In [3]:
abbrevs = pd.read_csv('data/state-abbrevs.csv')
areas = pd.read_csv('data/state-areas.csv')
population = pd.read_csv('data/state-population.csv')
display(abbrevs.head(), areas.head(), population.head())

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


合并pop与abbrevs两个DataFrame，分别依据state/region列和abbreviation列来合并。
为了保留所有信息，使用外合并

In [5]:
pop_abbr = pd.merge(population, abbrevs,
        left_on='state/region',
        right_on='abbreviation',
        how='outer')
pop_abbr.head()

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL


去除abbreviation的那一列（axis=1）

In [6]:
pop_abbr.drop('abbreviation', axis=1, inplace=True)
pop_abbr.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


查看存在缺失数据的列。
使用.isnull().any()，只有某一列存在一个缺失数据，就会显示True。

In [7]:
pop_abbr.isnull().any(axis=0)

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

查看缺失数据

In [8]:
pop_abbr.loc[:, pop_abbr.isnull().any(axis=0)]

,population,state
0,1117489.0,Alabama
1,4817528.0,Alabama
2,1130966.0,Alabama
3,4785570.0,Alabama
4,1125763.0,Alabama
5,4801627.0,Alabama
6,4757938.0,Alabama
7,1134192.0,Alabama
8,1111481.0,Alabama
9,4833722.0,Alabama


找到有哪些state/region使得state的值为NaN，使用unique()去重

In [16]:
pop_abbr.loc[pop_abbr.isnull().any(axis=1)]['state/region'].unique()

array(['PR', 'USA'], dtype=object)

为找到的这些state/region的state项补上正确的值，从而去除掉state这一列的所有NaN！
记住这样清除缺失数据NaN的方法！

In [22]:
update = {
    'USA': 'United states of America',
    'PR': 'PUERTO RICO'
}
# 修改列名
pop_abbr.rename(columns={'state/region': 'region'},
                inplace=True)

In [108]:
pa = pd.merge(abbrevs,population,
              left_on='abbreviation',
              right_on='state/region', how='outer')

In [109]:
pa.drop('abbreviation', axis=1, inplace=True)

In [110]:
pa.isnull().any(axis=0)

state            True
state/region    False
ages            False
year            False
population       True
dtype: bool

In [116]:
# 先查到某一列的 Series
# 通过： Series == '特征中的数据' 查询满足条件的所有索引数据(Series)
# 通过 查询结果的索引Series 作为.loc[行索引]，更新某一列的值
pa.loc[pa['state/region'] == 'USA', 'state'] = update['USA']
pa.loc[pa['state/region'] == 'PR', 'state'] = update['PR']

In [115]:
pa.loc[pa['state/region'] == 'USA']

,state,state/region,ages,year,population
2496,United states of America,USA,under18,1990,64218512.0
2497,United states of America,USA,total,1990,249622814.0
2498,United states of America,USA,total,1991,252980942.0
2499,United states of America,USA,under18,1991,65313018.0
2500,United states of America,USA,under18,1992,66509177.0
2501,United states of America,USA,total,1992,256514231.0
2502,United states of America,USA,total,1993,259918595.0
2503,United states of America,USA,under18,1993,67594938.0
2504,United states of America,USA,under18,1994,68640936.0
2505,United states of America,USA,total,1994,263125826.0


In [86]:
pop_abbr.head()

,region,ages,year,population,state
0,AL,under18,2012.0,1117489.0,Alabama
1,AL,total,2012.0,4817528.0,Alabama
2,AL,under18,2010.0,1130966.0,Alabama
3,AL,total,2010.0,4785570.0,Alabama
4,AL,under18,2011.0,1125763.0,Alabama


In [31]:
pa_copy = pop_abbr.query('region == "USA"')
pa_copy['state'] = 'United states of America'

pa_copy_2 = pop_abbr.query('region == "PR"')
pa_copy_2['state'] = 'PUERTO RICO'

# 使用有数据的DataFrame（右表）， 更新有Nan的DataFrame(左表)，
pop_abbr.update(pa_copy)
pop_abbr.update(pa_copy_2)

/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [34]:
pop_abbr.isnull().any(axis=0)

region        False
ages          False
year          False
population     True
state         False
dtype: bool

合并各州面积数据areas，使用左合并。
思考一下为什么使用左合并？

In [36]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [55]:
# 保留所有的州（左合并）
full = pd.merge(pop_abbr,areas, how='left')
full.head()

,region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012.0,1117489.0,Alabama,52423.0
1,AL,total,2012.0,4817528.0,Alabama,52423.0
2,AL,under18,2010.0,1130966.0,Alabama,52423.0
3,AL,total,2010.0,4785570.0,Alabama,52423.0
4,AL,under18,2011.0,1125763.0,Alabama,52423.0


继续寻找存在缺失数据的列

In [56]:
full.isnull().any(axis=0)

region           False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

我们会发现area(sq.mi)这一列有缺失数据，为了找出是哪一行，我们需要找出是哪个state没有数据

In [46]:
full.loc[full.isnull().any(axis=1)]['state'].unique()

array(['PUERTO RICO', 'United states of America'], dtype=object)

找出2010年的全民人口数据

In [58]:
q = 'year == 2010 & ages == "total" & region != "USA"'
full.query(q)['population'].sum()

313047503.0

对查询结果进行处理，以state列作为新的行索引

In [60]:
full.head()

,region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012.0,1117489.0,Alabama,52423.0
1,AL,total,2012.0,4817528.0,Alabama,52423.0
2,AL,under18,2010.0,1130966.0,Alabama,52423.0
3,AL,total,2010.0,4785570.0,Alabama,52423.0
4,AL,under18,2011.0,1125763.0,Alabama,52423.0


In [65]:
# 将列转成DataFrame的行索引
full.set_index('state', inplace=True)

In [67]:
full.head()

,region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,under18,2012.0,1117489.0,52423.0
Alabama,AL,total,2012.0,4817528.0,52423.0
Alabama,AL,under18,2010.0,1130966.0,52423.0
Alabama,AL,total,2010.0,4785570.0,52423.0
Alabama,AL,under18,2011.0,1125763.0,52423.0


计算人口密度。注意是Series/Series，其结果还是一个Series

In [81]:
full_pa = full.query('ages == "total" & region != "USA" & year == 2013')
full_pa['density'] = np.round(full_pa['population'] / full_pa['area (sq. mi)'], 2)


/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [82]:
full_pa.head()

,region,ages,year,population,area (sq. mi),density
state,,,,,,
Alabama,AL,total,2013.0,4833722.0,52423.0,92.21
Alaska,AK,total,2013.0,735132.0,656425.0,1.12
Arizona,AZ,total,2013.0,6626624.0,114006.0,58.13
Arkansas,AR,total,2013.0,2959373.0,53182.0,55.65
California,CA,total,2013.0,38332521.0,163707.0,234.15


排序，并找出2013年人口密度最高的五个州

In [84]:
full_pa.sort_values('density',
                    axis=0,
                    ascending=False)[:5]

,region,ages,year,population,area (sq. mi),density
state,,,,,,
District of Columbia,DC,total,2013.0,646449.0,68.0,9506.60
New Jersey,NJ,total,2013.0,8899339.0,8722.0,1020.33
Rhode Island,RI,total,2013.0,1051511.0,1545.0,680.59
Connecticut,CT,total,2013.0,3596080.0,5544.0,648.64
Massachusetts,MA,total,2013.0,6692824.0,10555.0,634.09


排序，并找出2013年人口密度最低的五个州

In [85]:
# 排序-按数值排序， by 根据axis轴的标签，进行升序或降序排序
full_pa.sort_values('density',
                    axis=0,
                    ascending=True)[:5]

,region,ages,year,population,area (sq. mi),density
state,,,,,,
Alaska,AK,total,2013.0,735132.0,656425.0,1.12
Wyoming,WY,total,2013.0,582658.0,97818.0,5.96
Montana,MT,total,2013.0,1015165.0,147046.0,6.90
North Dakota,ND,total,2013.0,723393.0,70704.0,10.23
South Dakota,SD,total,2013.0,844877.0,77121.0,10.96
